## Analyzing NewYork City Taxi Rides
### Author: Zynab Smaan.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, unix_timestamp, to_timestamp

spark = SparkSession \
    .builder \
    .appName("Analyzing New york city trips") \
    .getOrCreate()

In [2]:
trips = spark.read\
               .format("csv")\
               .option("header", "true")\
               .load("sample.csv")

In [3]:
trips.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_time_in_secs: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)



## Cleaning the data 

#### 1- Drop columns we don't need.

In [4]:
# Let's drop these columns
trips = trips.drop('medallion', 'vendor_id', 'rate_code', 'store_and_fwd_flag', 
                   'passenger_count', 'trip_distance')
trips.columns

['hack_license',
 'pickup_datetime',
 'dropoff_datetime',
 'trip_time_in_secs',
 'trip_distance',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude']

In [5]:
trips.show(1)

+--------------------+-------------------+-------------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|        hack_license|    pickup_datetime|   dropoff_datetime|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|
+--------------------+-------------------+-------------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|BA96DE419E711691B...|2013-01-01 15:11:48|2013-01-01 15:18:10|              382|         1.00|      -73.978165|      40.757977|       -73.989838|       40.751171|
+--------------------+-------------------+-------------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
only showing top 1 row



##### The size of the data before filteration.

In [20]:
trips.count()

99999

#### 2- Dropping nan values.

In [26]:
trips = trips.dropna()


99550

In [ ]:
trips.count()

#### 3- Creating duration (ms) column for each trip.

In [29]:
# Create column duration 
trips = trips.withColumn("duration", unix_timestamp("dropoff_datetime", "yyyy-MM-dd HH:mm:ss")
                         .cast("double") - unix_timestamp("pickup_datetime", "yyyy-MM-dd HH:mm:ss")
                         .cast("double"))

In [30]:
trips.show(5)

+--------------------+-------------------+-------------------+-----------------+-------------+----------------+---------------+-----------------+----------------+--------+
|        hack_license|    pickup_datetime|   dropoff_datetime|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|duration|
+--------------------+-------------------+-------------------+-----------------+-------------+----------------+---------------+-----------------+----------------+--------+
|BA96DE419E711691B...|2013-01-01 15:11:48|2013-01-01 15:18:10|              382|         1.00|      -73.978165|      40.757977|       -73.989838|       40.751171|   382.0|
|9FD8F69F0804BDB55...|2013-01-06 00:18:35|2013-01-06 00:22:54|              259|         1.50|      -74.006683|      40.731781|       -73.994499|        40.75066|   259.0|
|9FD8F69F0804BDB55...|2013-01-05 18:49:41|2013-01-05 18:54:23|              282|         1.10|      -74.004707|       40.73777|       -74.00

#### 4- filter data if the duration is less than 0 or bigger than (4 hours = 4*60*60 = 14400).

In [31]:
# filter data if the duration is less than 0 or bigger than (4 hours = 4*60*60 = 14400)
trips = trips.filter((trips['duration'] <= 14400.0) & (trips['duration'] > 0.0))

##### The size of the data after filteration.

In [33]:
trips.count()

99550

**NOTES**
 - The data is cleaned, and we have columns we need.
 - The size of the data is reduced now w.r.t our cleaning.